In [ ]:
arff.load

In [ ]:
import torch

import copy
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import arff
from sklearn.model_selection import train_test_split

from torch import nn, optim

import torch.nn.functional as F
#from arff2pandas import a2p


%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In this tutorial, you'll learn how to detect anomalies in Time Series data using an LSTM Autoencoder. You're going to use real-world ECG data from a single patient with heart disease to detect abnormal hearbeats.

- [Read the tutorial](https://www.curiousily.com/posts/time-series-anomaly-detection-using-lstm-autoencoder-with-pytorch-in-python/)
- [Run the notebook in your browser (Google Colab)](https://colab.research.google.com/drive/1_J2MrBSvsJfOcVmYAN2-WSp36BtsFZCa)
- [Read the Getting Things Done with Pytorch book](https://github.com/curiousily/Getting-Things-Done-with-Pytorch)

By the end of this tutorial, you'll learn how to:

- Prepare a dataset for Anomaly Detection from Time Series Data
- Build an LSTM Autoencoder with PyTorch
- Train and evaluate your model
- Choose a threshold for anomaly detection
- Classify unseen examples as normal or anomaly

In [ ]:
#########################################

## 1-  Get the Data



The [dataset](http://timeseriesclassification.com/description.php?Dataset=ECG5000) contains 5,000 Time Series examples (obtained with ECG) with 140 timesteps. Each sequence corresponds to a single heartbeat from a single patient with congestive heart failure.

> An electrocardiogram (ECG or EKG) is a test that checks how your heart is functioning by measuring the electrical activity of the heart. With each heart beat, an electrical impulse (or wave) travels through your heart. This wave causes the muscle to squeeze and pump blood from the heart. [Source](https://www.heartandstroke.ca/heart/tests/electrocardiogram)

We have 5 types of hearbeats (classes):

- Normal (N) 
- R-on-T Premature Ventricular Contraction (R-on-T PVC)
- Premature Ventricular Contraction (PVC)
- Supra-ventricular Premature or Ectopic Beat (SP or EB) 
- Unclassified Beat (UB).

In [ ]:
!gdown --id 16MIleqoIr1vYxlGk4GKnGmrsCPuWkkpT

In [ ]:
!unzip -qq ECG5000.zip

In [ ]:
! pip install arff

In [ ]:
import torch
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
import copy
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import arff
from sklearn.model_selection import train_test_split

The data comes in multiple formats. We'll load the `arff` files into Pandas data frames:

We'll combine the training and test data into a single data frame. This will give us more data to train our Autoencoder. We'll also shuffle it:

In [ ]:
import pandas as pd
from scipy.io.arff import loadarff 

raw_data = loadarff('/content/ECG5000_TRAIN.arff')
df_data = pd.DataFrame(raw_data[0])
raw_data_test = loadarff('/content/ECG5000_TEST.arff')
df_data_test = pd.DataFrame(raw_data_test[0])

In [ ]:
df = pd.concat([df_data ,df_data_test ] ,axis = 0)

In [ ]:
df

We have 5,000 examples. Each row represents a single heartbeat record. Let's name the possible classes:

In [ ]:
CLASS_NORMAL = 1

class_names = ['Normal','R on T','PVC','SP','UB']

## Exploratory Data Analysis

Let's check how many examples for each heartbeat class do we have:

In [ ]:
df.target =  df.target.apply(lambda x : str(x)[2] )

In [ ]:
df.target.value_counts()

Let's plot the results:

In [ ]:
import seaborn as sns
ax = sns.countplot(df.target)
ax.set_xticklabels(class_names);

The normal class, has by far, the most examples. This is great because we'll use it to train our model.

Let's have a look at an averaged (smoothed out with one standard deviation on top and bottom of it) Time Series for each class:

In [ ]:
def plot_time_series_class(data, class_name, ax, n_steps=10):
  time_series_df = pd.DataFrame(data)

  smooth_path = time_series_df.rolling(n_steps).mean()
  path_deviation = 2 * time_series_df.rolling(n_steps).std()

  under_line = (smooth_path - path_deviation)[0]
  over_line = (smooth_path + path_deviation)[0]

  ax.plot(smooth_path, linewidth=2)
  ax.fill_between(
    path_deviation.index,
    under_line,
    over_line,
    alpha=.125
  )
  ax.set_title(class_name)

In [ ]:
classes = df.target.unique()

fig, axs = plt.subplots(
  nrows=len(classes) // 3 + 1,
  ncols=3,
  sharey=True,
  figsize=(14, 8)
)

for i, cls in enumerate(classes):
  ax = axs.flat[i]
  data = df[df.target == cls] \
    .drop(labels='target', axis=1) \
    .mean(axis=0) \
    .to_numpy()
  plot_time_series_class(data, class_names[i], ax)

fig.delaxes(axs.flat[-1])
fig.tight_layout();

It is very good that the normal class has a distinctly different pattern than all other classes. Maybe our model will be able to detect anomalies?

### Data Preprocessing

Let's get all normal heartbeats and drop the target (class) column:

In [ ]:
# we need to shuffle the data before
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
normal_df = df[df.target == str(CLASS_NORMAL)].drop(labels='target', axis=1)
normal_df.shape

We'll merge all other classes and mark them as anomalies:

In [ ]:
anomaly_df = df[df.target != str(CLASS_NORMAL)].drop(labels='target', axis=1)
anomaly_df.shape

We'll split the normal examples into train, validation and test sets:

In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(
  normal_df,
  test_size=0.15,
  random_state=RANDOM_SEED
)

val_df, test_df = train_test_split(
  val_df,
  test_size=0.33, 
  random_state=RANDOM_SEED
)

We need to convert our examples into sequences, so we can use them to train our Autoencoder. Let's write a helper function for that:

In [ ]:
import numpy as np
def create_dataset(df):

  sequences = np.array(df.astype(np.float32).to_numpy().tolist())

  sequences = np.expand_dims(sequences , axis=2)
  return sequences #

In [ ]:
train_dataset = create_dataset(train_df)
val_dataset= create_dataset(val_df)
test_normal_dataset = create_dataset(test_df)
test_anomaly_dataset = create_dataset(anomaly_df)

In [ ]:
train_dataset.shape

# Build LSTM autoencoder

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Input, Dropout
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(train_dataset.shape[1], train_dataset.shape[2])))
model.add(Dropout(rate=0.2))

model.add(RepeatVector(train_dataset.shape[1]))

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(TimeDistributed(Dense(train_dataset.shape[2])))
model.compile(optimizer='adam', loss='mae')
model.summary()

In [ ]:
history = model.fit(train_dataset, train_dataset, epochs=10, batch_size=32, validation_split=0.1, verbose=1)

In [ ]:
epochs = [i for i in range(10)]
plt.plot(epochs , history.history['loss'] , 'g-o' , label = 'Training Loss')
plt.plot(epochs , history.history['val_loss'] , 'r-o' , label = 'Validation Loss')
plt.legend()
plt.show()

# compute recounstruction loss 

In [ ]:
def sequence_prediction(data ):
  pre = model.predict(data )
  trainMAE = np.mean(np.abs(pre - data), axis=1)
  return trainMAE    

In [ ]:
preictions_normal_losses = sequence_prediction(test_normal_dataset )

In [ ]:
sns.distplot(preictions_normal_losses, bins=50, kde=True)

In [ ]:
def sequence_prediction_(data , threshold):
  outputs = []
  pre = model.predict(data )
  trainMAE = np.mean(np.abs(pre - data), axis=1)
  for i in range(len(trainMAE)) :
    if trainMAE[i] > threshold : 
      outputs.append('anomaly')
    else : 
      outputs.append('normal')
  return outputs 

In [ ]:
preictions_normal = sequence_prediction_(test_normal_dataset , 0.4 )

In [ ]:
preictions_normal

In [ ]:
preictions_anomaly = sequence_prediction_(test_anomaly_dataset , 0.2)

In [ ]:
preictions_anomaly 

Our model converged quite well. Seems like we might've needed a larger validation set to smoothen the results, but that'll do for now.

## Saving the model

Let's store the model for later use:

In [ ]:
model.save('/content/anomaly_detection.h5')